# Soccer Computer Vision Project

With the models in the `models` folder that we trained using the notebooks in the `training` folder, we will detect players, referees, and the ball on the field and assign players to their teams.

This was ran on a GPU P100 on Kaggle.

In [1]:
!pip install -q ultralytics supervision umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 910.1/910.1 kB 15.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.0 MB/s eta 0:00:00


In [2]:
import supervision as sv
from ultralytics import YOLO
import torch
import umap
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer, util
import numpy as np
from tqdm import tqdm
from more_itertools import chunked

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Load Model
model = YOLO("models/best.pt")

INPUT_VIDEO_PATH = "videos/input/121364_0.mp4"
OUTPUT_VIDEO_PATH = "videos/output/121364_0_out.mp4"

In [4]:
# CLIP VITB23 TRANSFORMERS
class ClipTeamClassifier:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.embeddings_model = SentenceTransformer('clip-ViT-B-32').to(self.device)
        self.reducer = umap.UMAP(n_components=3)
        self.clustering_model = KMeans(n_clusters=2)

    def fit(self, crops):
        data = self.get_embeddings(crops)
        projections = self.reducer.fit_transform(data)
        self.clustering_model.fit(projections)

    def get_embeddings(self, crops):
        crops = [sv.cv2_to_pillow(crop) for crop in crops]
        data = []
        batches = chunked(crops, 32)
        with torch.no_grad():
            for batch in batches:
                data.append(self.embeddings_model.encode(batch))
        return np.concatenate(data)

    def predict(self, crops):
        if len(crops) == 0:
            return np.array([])

        data = self.get_embeddings(crops)
        projections = self.reducer.transform(data)
        return self.clustering_model.predict(projections)

In [5]:
# Get GK Teams
def get_gk_teams(gk_detections, player_detections):
    players_0_xy = player_detections[player_detections.class_id == 0].get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    players_1_xy = player_detections[player_detections.class_id == 1].get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    team_0_centroid = players_0_xy.mean(axis=0)
    team_1_centroid = players_1_xy.mean(axis=0)
    gk_teams = []
    for gk_xy in gk_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER):
        if np.linalg.norm(gk_xy - team_0_centroid) < np.linalg.norm(gk_xy - team_1_centroid):
            gk_teams.append(0)
        else:
            gk_teams.append(1)
    return np.array(gk_teams).astype(int)

In [7]:
#Main


# Video info
video_info = sv.VideoInfo.from_video_path(INPUT_VIDEO_PATH)
video_sink = sv.VideoSink(target_path=OUTPUT_VIDEO_PATH, video_info=video_info)

# Tracker
tracker = sv.ByteTrack()
tracker.reset()

# Process the video
frame_generator = sv.get_video_frames_generator(source_path=INPUT_VIDEO_PATH, stride=30)
crops = []
for frame in tqdm(frame_generator, desc='collecting crops'):
    result = model.predict(frame, conf=0.3)[0]
    detections = sv.Detections.from_ultralytics(result)
    detections = detections.with_nms(threshold=0.5, class_agnostic=True)
    detections = detections[detections.class_id == 2]
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in detections.xyxy]
    crops += players_crops
team_class = ClipTeamClassifier()
team_class.fit(crops)

frame_gen = sv.get_video_frames_generator(source_path=INPUT_VIDEO_PATH)
#get annotators
ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)

tracks = {}
teams = {}

initial_crops = []
with video_sink:
    for frame in tqdm(frame_gen, total=video_info.total_frames):
        # Perform detection
        detections = model.predict(frame, conf=0.1)
        detection_supervision = sv.Detections.from_ultralytics(detections[0])

        ball_detections = detection_supervision[detection_supervision.class_id == 0]

        all_detections = detection_supervision[detection_supervision.class_id != 0]
        all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
        all_detections.class_id -= 1
        all_detections = tracker.update_with_detections(all_detections)

        #class_id = 3
        #tracker_id = 4

        new_cls_ids = []
        for track in all_detections:
            if track[4] not in tracks:
                tracks[track[4]] = {}
            if track[3] not in tracks[track[4]]:
                tracks[track[4]][track[3]] = 0
            tracks[track[4]][track[3]] += 1
            if 0 in tracks[track[4]] and tracks[track[4]][0] > 5 :
                new_cls_ids.append(0)
            elif 2 in tracks[track[4]] and tracks[track[4]][2] > 5 :
                new_cls_ids.append(2)
            elif 1 in tracks[track[4]] and tracks[track[4]][1] > 5 :
                new_cls_ids.append(1)
            else:
                new_cls_ids.append(track[3])
        all_detections.class_id = np.array(new_cls_ids)

        TEAM_ZERO = 0
        TEAM_ONE = 1

        player_detections = all_detections[all_detections.class_id == 1]
        gk_detections = all_detections[all_detections.class_id == 0]
        ref_detections = all_detections[all_detections.class_id == 2]

        crops = [sv.crop_image(frame, xyxy) for xyxy in player_detections.xyxy]
        player_detections.class_id = team_class.predict(crops)

        gk_detections.class_id = get_gk_teams(gk_detections, player_detections)

        all_detections = sv.Detections.merge([player_detections, gk_detections, ref_detections])

        labels = [
            f"#{tracker_id}"
            for tracker_id
            in all_detections.tracker_id
        ]

        annotated_frame = frame.copy()
        annotated_frame = ellipse_annotator.annotate(
            scene=annotated_frame,
            detections=all_detections)
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame,
            detections=all_detections,
            labels=labels)
        annotated_frame = triangle_annotator.annotate(
            scene=annotated_frame,
            detections=ball_detections)
        # Write the frame to the output video
        video_sink.write_frame(annotated_frame)

collecting crops: 0it [00:00, ?it/s]


0: 736x1280 1 ball, 1 goalkeeper, 22 players, 2 referees, 71.7ms
Speed: 8.3ms preprocess, 71.7ms inference, 1.7ms postprocess per image at shape (1, 3, 736, 1280)


collecting crops: 1it [00:00,  6.45it/s]


0: 736x1280 1 goalkeeper, 21 players, 2 referees, 72.0ms
Speed: 10.2ms preprocess, 72.0ms inference, 1.6ms postprocess per image at shape (1, 3, 736, 1280)


collecting crops: 2it [00:00,  4.80it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|█████████▉| 749/750 [03:21<00:00,  3.85it/s]


0: 736x1280 1 goalkeeper, 21 players, 2 referees, 65.7ms
Speed: 6.3ms preprocess, 65.7ms inference, 1.5ms postprocess per image at shape (1, 3, 736, 1280)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 750/750 [03:22<00:00,  3.71it/s]
